In [8]:
import sigpde.torch as sig
import torch
import math
device = torch.device('cuda:0')

In [16]:
kernel = sig.SigPDE(sig.kernels.LinearKernel(), 4)

In [10]:
def generate(batch_size, length, dimension, device = torch.device('cpu')):
  random_walks = torch.randn(batch_size, length, dimension, dtype = torch.double, device = device) / math.sqrt(length)
  start = torch.zeros([batch_size, 1, dimension], device=device, dtype=torch.double)
  random_walks = torch.cat((start, random_walks), dim=1)
  random_walks = torch.cumsum(random_walks, dim=1)
  return random_walks

In [11]:
x = generate(4, 512, 10, device=device)
y = generate(4, 512, 10, device=device)

In [17]:
kernel.pairwise(x, x)

tensor([ 5379.4125,  7641.2184, 27262.1215,  4688.3723], device='cuda:0',
       dtype=torch.float64)